# Real Agent Integration with Agentune Simulate

This notebook demonstrates how to integrate a real agent system with the Agentune Simulate library. You'll learn to:

- Implement a custom agent by extending the `Agent` interface
- Integrate your custom agent with the simulation framework
- Run simulations with real agents and simulated customers
- Compare results between real and simulated agents

## Use Case: Real Agent + Simulated Customer

This pattern is useful for:
- Testing your actual agent system against various customer scenarios
- Evaluating agent performance without human customers
- A/B testing between different agent implementations
- Stress testing your agent with diverse conversation patterns

## Part 1: Setup and Imports

In [ ]:
import os
import getpass
import asyncio
from typing import Optional, override
from datetime import datetime

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from agentune.simulate.models import Conversation, Message, Outcomes
from agentune.simulate.participants.agent.base import Agent
from agentune.simulate.participants.factories.base import AgentFactory
from agentune.simulate.rag import conversations_to_langchain_documents
from agentune.simulate.simulation.session_builder import SimulationSessionBuilder
from utils import setup_logging_and_asyncio, load_data_with_outcomes

In [ ]:
# Load sample data
conversations, outcomes_tuple = load_data_with_outcomes("data/sample_conversations.csv")
outcomes = Outcomes(outcomes=outcomes_tuple)

# Setup models and vector store for the simulated customer
chat_model = ChatOpenAI(model="gpt-4o", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
documents = conversations_to_langchain_documents(conversations)
vector_store = InMemoryVectorStore.from_documents(documents, embeddings)

print(f"✓ Loaded {len(conversations)} conversations with vector store")

## Part 2: Custom Real Agent Implementation

Here we'll create a `MockRealAgent` that simulates calling an external agent system. In practice, this would make HTTP requests, call APIs, or interface with your actual agent platform.

In [ ]:
class MockRealAgent(Agent):
    """A minimal real agent that simulates external agent system integration."""

    def __init__(self):
        self._messages = [
            "Hello! How can I help you today?",
            "I understand your concern. Let me help you with that issue.",
            "Thank you for contacting us. Have a great day!"
        ]
        self._count = 0

    async def get_next_message(self, conversation: Conversation) -> Optional[Message]:
        """Generate agent response - replace this with your actual agent API call."""

        if self._count >= len(self._messages):
            return None  # No more responses. None indicates the agent is not responding.

        agent_response = self._messages[self._count]
        self._count += 1

        return Message(
            sender=self.role,
            content=agent_response,
            timestamp=datetime.now()
        )

print("✓ MockRealAgent class implemented")

In [ ]:
# Create a factory for our real agent
class MockRealAgentFactory(AgentFactory):
    """Factory for creating MockRealAgent instances."""

    def create_participant(self) -> Agent:
        return MockRealAgent()

## Part 3: Simulation Session with Real Agent

In [ ]:
# Create simulation session with real agent + simulated customer
real_agent_factory = MockRealAgentFactory()

session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=8
).with_agent_factory(real_agent_factory).build()

print("✓ Simulation session created with real agent")

In [ ]:
# Run simulation with real agent
base_conversations = conversations[:3]  # Use first 3 conversations as scenarios
real_agent_result = await session.run_simulation(real_conversations=base_conversations)

print("✓ Real agent simulation completed!")
print(f"Simulated {len(real_agent_result.simulation_results)} conversations")

## Part 4: Results Analysis and Comparison

Let's compare our real agent results with a standard RAG-based simulated agent.

In [ ]:
# Run comparison with standard RAG agent
standard_session = SimulationSessionBuilder(
    default_chat_model=chat_model,
    outcomes=outcomes,
    vector_store=vector_store,
    max_messages=8
).build()  # Uses default RAG-based agent

standard_result = await standard_session.run_simulation(real_conversations=base_conversations)

print("✓ Standard agent simulation completed!")

In [ ]:
# Compare results
print("========================================")
print("REAL AGENT vs SIMULATED AGENT COMPARISON")
print("========================================")

print("\n--- REAL AGENT RESULTS ---")
print(real_agent_result.generate_summary())

print("\n--- SIMULATED AGENT RESULTS ---")
print(standard_result.generate_summary())

# Save results
real_agent_result.save_to_file("real_agent_results.json")
standard_result.save_to_file("simulated_agent_results.json")

print("\n✓ Results saved to files")

In [ ]:
# Show sample conversation with real agent
print("========================================")
print("SAMPLE REAL AGENT CONVERSATION")
print("========================================")

if real_agent_result.simulation_results:
    sample_conversation = real_agent_result.simulation_results[0].conversation
    
    for i, message in enumerate(sample_conversation.messages, 1):
        print(f"{i}. {message.sender}: {message.content[:100]}...")
        if i >= 6:  # Show first 6 messages
            break
    
    if len(sample_conversation.messages) > 6:
        print(f"... and {len(sample_conversation.messages) - 6} more messages")
else:
    print("No simulation results to display")

## Next Steps

You've successfully integrated a real agent with the Agentune Simulate framework! Here's how to extend this for your use case:

### 1. **Implement Your Real Agent**
Replace the `MockRealAgent` with your actual agent implementation:

```python
class YourRealAgent(Agent):
    async def get_next_message(self, conversation: Conversation) -> Optional[Message]:
        # Make API call to your agent system
        response = await your_agent_api.get_response(conversation)
        return Message(sender="agent", content=response, timestamp=datetime.now())
```

### 2. **Testing and Validation**
- Use this framework to test your agent against diverse scenarios
- Compare performance metrics between different agent versions
- Validate agent behavior with adversarial testing

### 3. **Production Considerations**
- Monitor agent response times and success rates
- Implement graceful degradation for agent failures
- Scale simulations with concurrent conversation limits

The framework provides a flexible foundation for integrating any agent system while maintaining compatibility with all existing simulation features.